In [1]:
from sklearn import datasets

iris_data = datasets.load_iris()
#print(iris_data.data)
#print(iris_data.target)
iris_data = datasets.load_iris()
dataSet=[]
for i,example in enumerate(iris_data.data):
    data = list(example)
    data.append(iris_data.target[i])
    #print(data)
    dataSet.append(data)
print(dataSet)

[[5.1, 3.5, 1.4, 0.2, 0], [4.9, 3.0, 1.4, 0.2, 0], [4.7, 3.2, 1.3, 0.2, 0], [4.6, 3.1, 1.5, 0.2, 0], [5.0, 3.6, 1.4, 0.2, 0], [5.4, 3.9, 1.7, 0.4, 0], [4.6, 3.4, 1.4, 0.3, 0], [5.0, 3.4, 1.5, 0.2, 0], [4.4, 2.9, 1.4, 0.2, 0], [4.9, 3.1, 1.5, 0.1, 0], [5.4, 3.7, 1.5, 0.2, 0], [4.8, 3.4, 1.6, 0.2, 0], [4.8, 3.0, 1.4, 0.1, 0], [4.3, 3.0, 1.1, 0.1, 0], [5.8, 4.0, 1.2, 0.2, 0], [5.7, 4.4, 1.5, 0.4, 0], [5.4, 3.9, 1.3, 0.4, 0], [5.1, 3.5, 1.4, 0.3, 0], [5.7, 3.8, 1.7, 0.3, 0], [5.1, 3.8, 1.5, 0.3, 0], [5.4, 3.4, 1.7, 0.2, 0], [5.1, 3.7, 1.5, 0.4, 0], [4.6, 3.6, 1.0, 0.2, 0], [5.1, 3.3, 1.7, 0.5, 0], [4.8, 3.4, 1.9, 0.2, 0], [5.0, 3.0, 1.6, 0.2, 0], [5.0, 3.4, 1.6, 0.4, 0], [5.2, 3.5, 1.5, 0.2, 0], [5.2, 3.4, 1.4, 0.2, 0], [4.7, 3.2, 1.6, 0.2, 0], [4.8, 3.1, 1.6, 0.2, 0], [5.4, 3.4, 1.5, 0.4, 0], [5.2, 4.1, 1.5, 0.1, 0], [5.5, 4.2, 1.4, 0.2, 0], [4.9, 3.1, 1.5, 0.1, 0], [5.0, 3.2, 1.2, 0.2, 0], [5.5, 3.5, 1.3, 0.2, 0], [4.9, 3.1, 1.5, 0.1, 0], [4.4, 3.0, 1.3, 0.2, 0], [5.1, 3.4, 1.5, 0.2, 0],

In [2]:
from sklearn import datasets
def createDataSet():
    iris_data = datasets.load_iris()
    dataSet=[]
    for i,example in enumerate(iris_data.data):
        data = list(example)
        data.append(iris_data.target[i])
        #print(data)
        dataSet.append(data)
    labels=list(set(iris_data.target))
    return dataSet,labels

In [3]:
from math import log

def calcShannonEnt(dataSet):
    numEntries = len(dataSet) #nrows
    #为所有的分类类目创建字典
    labelCounts ={}
    for featVec in dataSet:
        currentLable=featVec[-1] #取得最后一列数据
        if currentLable not in labelCounts.keys():
            labelCounts[currentLable]=0
        labelCounts[currentLable]+=1
    #计算香农熵
    shannonEnt=0.0
    for key in labelCounts:
        prob = float(labelCounts[key]) / numEntries
        shannonEnt -= prob * log(prob, 2)
    return shannonEnt

In [4]:
#定义按照某个特征进行划分的函数splitDataSet
#输入三个变量（待划分的数据集，特征，分类值）
def splitDataSet(dataSet,axis,value):
    retDataSet=[]
    retDataSet1=[]
    for featVec in dataSet:
        if featVec[axis]<=value : 
            reduceFeatVec=featVec[:axis]
            reduceFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reduceFeatVec)
        else: 
            reduceFeatVec=featVec[:axis]
            reduceFeatVec.extend(featVec[axis+1:])
            retDataSet1.append(reduceFeatVec)
    return retDataSet,retDataSet1 #返回不含划分特征的子集

#定义按照最大信息增益划分数据的函数
def chooseBestFeatureToSplit(dataSet):
    numFeature=len(dataSet[0])-1
    baseEntropy=calcShannonEnt(dataSet)#香农熵
    bestInforGain=0
    bestFeature=-1
    bestValue=-1
    for i in range(numFeature):
        featList=[number[i] for number in dataSet] #得到某个特征下所有值（某列）
        uniqualVals=set(featList) #set无重复的属性特征值
        newEntropy=0
        for value in uniqualVals:
            subDataSet,subDataSet1=splitDataSet(dataSet,i,value)
            prob=len(subDataSet)/float(len(dataSet)) #即p(t)
            prob1=len(subDataSet1)/float(len(dataSet))
            newEntropy=prob*calcShannonEnt(subDataSet)+prob1*calcShannonEnt(subDataSet1)#对各子集香农熵求和
            infoGain=baseEntropy-newEntropy #计算信息增益
        #最大信息增益
            if (infoGain>bestInforGain):
                bestInforGain=infoGain
                bestFeature=i
                bestValue=value
    return bestFeature,bestValue #返回特征值

In [5]:
import operator
#投票表决代码
def majorityCnt(classList):
    classCount={}
    for vote in classList:
        if vote not in classCount.keys():
            classCount[vote]=0
        classCount[vote]+=1
    sortedClassCount=sorted(classCount.items(), key = operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]

def createTree(dataSet,labels):
    classList=[example[-1] for example in dataSet]
    #类别相同，停止划分
    if classList.count(classList[0])==len(classList):
        return classList[0]
    #长度为1，返回出现次数最多的类别
    if len(dataSet[0])==1:
        return majorityCnt(classList)
    #按照信息增益最高选取分类特征属性
    bestFeat,bestVal=chooseBestFeatureToSplit(dataSet)#返回分类的特征序号
    bestFeatLable=labels[bestFeat] #该特征的label
    myTree={bestFeatLable:{}} #构建树的字典
    del(labels[bestFeat]) #从labels的list中删除该label
    subLables=labels[:] #子集合
        #构建数据的子集合，并进行递归
    myTree[bestFeatLable]['<='+str(bestVal)]=createTree(splitDataSet(dataSet,bestFeat,bestVal)[0],subLables)
    subLables=labels[:] 
    myTree[bestFeatLable]['>'+str(bestVal)]=createTree(splitDataSet(dataSet,bestFeat,bestVal)[1],subLables)
    return myTree

In [6]:
#输入三个变量（决策树，属性特征标签，测试的数据）
def classify(inputTree,featLables,testVec):
    firstStr=list(inputTree.keys())[0] #获取树的第一个特征属性
    secondDict=inputTree[firstStr] #树的分支，子集合Dict
    featIndex=featLables.index(firstStr) #获取决策树第一层在featLables中的位置
    for key in secondDict.keys():
        if '<=' in key:
            val = float(key[2:])
            if testVec[featIndex]<=val:
                if type(secondDict[key]).__name__=='dict':
                    classLabel=classify(secondDict[key],featLables,testVec)
                else:classLabel=secondDict[key]
        else:
            val = float(key[1:])
            if testVec[featIndex]>val:
                if type(secondDict[key]).__name__=='dict':
                    classLabel=classify(secondDict[key],featLables,testVec)
                else:classLabel=secondDict[key]
    return classLabel

In [7]:
from sklearn.model_selection import train_test_split
myDat, labels = createDataSet()
data_train, data_test= \
    train_test_split(myDat, test_size = 0.2, random_state = 42)
myTree = createTree(data_train,['spealLength', 'spealwidth', 'setalLength', 'setalLength']) 
print(myTree)
feaList = ['spealLength', 'spealwidth', 'setalLength', 'setalLength']
count = 0
for testData in data_test:
    dic = classify(myTree, feaList, testData[:-1])
    if dic == testData[-1]:
        count+=1
accuracy = count/len(data_test)
print(accuracy)

{'setalLength': {'>1.9': {'setalLength': {'>1.7': {'spealLength': {'<=5.9': {'spealwidth': {'>3.0': 1, '<=3.0': 2}}, '>5.9': 2}}, '<=1.7': {'spealLength': {'<=7.0': {'spealwidth': {'<=2.8': 1, '>2.8': 1}}, '>7.0': 2}}}}, '<=1.9': 0}}
0.7
